1. Thu thập các đường dẫn thuộc mỗi chuyên mục từ trang web https://vnexpress.net/

- Viết hàm lấy tất cả các đường dẫn của bài báo trong trang web

In [14]:
# Gọi thư viện
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc:
URL = 'https://vnexpress.net/'

def _get_links(url):
    links = [] 
    html = urlopen(url).read()  
    soup = BeautifulSoup(html, 'html.parser') 

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'https://vnexpress.net/[^\s"]+\.html', str(soup)))  

    for url in urls:
        # Chỉ lấy đường dẫn là trang web nhiều bài báo
        if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia'): 
            links.append(url)

    return links


- Kết quả trả về của hàm với đường dẫn gốc của VNexpress

In [15]:
_get_links(URL)

['https://vnexpress.net/tp-hcm-se-xay-nut-giao-4-tang-o-cua-ngo-dong-bac-4804568.html',
 'https://vnexpress.net/luong-giao-vien-bao-nhieu-la-du-4804505.html',
 'https://vnexpress.net/quan-nhan-my-den-israel-van-hanh-la-chan-ten-lua-4804608.html',
 'https://vnexpress.net/con-sot-tui-mu-cua-gioi-tre-viet-4802141.html',
 'https://vnexpress.net/de-nghi-lam-ro-muc-dau-tu-hon-67-ty-usd-duong-sat-toc-do-cao-bac-nam-4804562.html',
 'https://vnexpress.net/ten-lua-manh-nhat-the-gioi-starship-co-gi-dac-biet-4804521.html',
 'https://vnexpress.net/hai-vu-tai-nan-oto-tren-quoc-lo-o-binh-thuan-dong-nai-4804618.html',
 'https://vnexpress.net/tac-gia/bui-khanh-nguyen-1950.html',
 'https://vnexpress.net/pho-thu-linh-hezbollah-ngung-ban-la-giai-phap-duy-nhat-4804577.html',
 'https://vnexpress.net/cuoc-tap-kich-uav-phoi-bay-diem-yeu-phong-khong-israel-4804189.html',
 'https://vnexpress.net/thu-tuong-phan-dau-moi-nam-co-9-10-trieu-tan-gao-chat-luong-cao-4804570.html',
 'https://vnexpress.net/no-lo-hoi-lam-

- Lấy đường dẫn của các bài toán thuộc từng chuyên mục

In [18]:
# Danh sách các chuyên mục
categories = [
    'thoi-su', 'kinh-doanh', 'khoa-hoc', 'giai-tri',
    'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe',
    'doi-song', 'du-lich', 'so-hoa', 'xe'
]

# Danh sách các đường dẫn đến các bài báo của mỗi chuyên mục
article_url = {}

for category in categories:
    article_url[category] = []

    # Lấy 5 trang trong mỗi chuyên mục
    for i in range(1, 11):
        url = '{}/{}-p{}'.format(URL, category, str(i))
        article_url[category] += _get_links(url)

    # Loại bỏ đường dẫn trùng nhau
    article_url[category] = list(set(article_url[category]))

    # Hiển thị số đường dẫn thu thập được trong chuyên mục
    print(category, len(article_url[category]))



thoi-su 27
kinh-doanh 27
khoa-hoc 27
giai-tri 27
the-thao 27
phap-luat 27
giao-duc 27
suc-khoe 27
doi-song 27
du-lich 27
so-hoa 27
xe 27


In [19]:
# Lấy riêng các đường dẫn trong chuyên mục theo góc nhìn từng mục con
goc_nhin = [
    'binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-su-khoe',
    'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong',
    'van-hoa-loi-song', 'covid-19'
]

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
    url = '{}/{}/{}'.format(URL, 'goc-nhin', sub_cate)
    article_url['goc-nhin'] += _get_links(url)

# Loại bỏ đường dẫn trùng nhau trong chuyên mục 'goc-nhin'
article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))

goc-nhin 27


2. Lấy nội dung của bài báo ứng với từng đường dẫn cụ thể và lưu vào một file trong folder chuyên mục tương ứng.

- Viết hàm lấy nội dung của bài báo ứng với đường dẫn cụ thể

In [20]:
def _get_content(url):
    # Nội dung bài báo
    content = ''
    
    # Mở URL và đọc nội dung trang
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy phần chứa nội dung bài báo
    div_content = soup.select('.page-detail .container')
    if len(div_content) > 0:
        div_content = div_content[0]

        # Lấy phần mô tả
        description = div_content.find_all('p', {'class': 'description'})
        if len(description) > 0:
            text_description = description[-1].get_text()
            location = description[-1].find('span', {'class': 'location-stamp'})
            if location is not None:
                content = text_description[len(location.get_text()):]
            else:
                content = text_description   

        # Lấy phần chứa nội dung chi tiết bài báo
        detail = div_content.find('article', {'class': 'fck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    # Kiểm tra đoạn văn bản không rỗng, không bắt đầu bằng dấu '>>' và kết thúc bằng các ký tự hợp lệ
                    if p_text != '' and not p_text.startswith('>>') and p_text[-1] in ['.', '!', '?']:
                        content += ' ' + p_text

    # Xóa ký tự ngắt dòng và trả về
    return re.sub(r'\n', '', content)

+ Kết quả trả về của hàm với đường dẫn dầu tiên trong chuyên mục số hóa

In [21]:
_get_content(article_url['giao-duc'][0])

'Ngoài tuyến đường sắt chạy qua, nút giao Vành đai 2 - Phạm Văn Đồng, TP Thủ Đức, sẽ được xây dựng hầm chui, cầu vượt hai tầng giúp xe thuận lợi rẽ đi các hướng. Phương án thiết kế nút giao vừa được chính quyền TP HCM chấp thuận theo đề xuất của Sở Giao thông Vận tải. Đây là nút giao thuộc phạm vi dự án xây dựng đoạn 2 của Vành đai 2 (từ đại lộ Võ Nguyên Giáp đến Phạm Văn Đồng - dài 2,8 km, tổng vốn hơn 4.500 tỷ đồng). Nếu tính cả tuyến đường sắt hiện hữu chạy qua, nút giao sẽ có quy mô 4 tầng. Trong đó, tầng một có hai cầu vượt: cầu số 1 gồm hai nhánh (6 làn xe) nằm trên Vành đai 2, vượt qua đường Linh Đông, Phạm Văn Đồng, Kha Vạn Cân, rạch Ngang; cầu số 2 có hai làn xe, rẽ trái từ Vành đai 2 xuống đường Phạm Văn Đồng, hướng về sân bay Tân Sơn Nhất. Phía trên tầng này sẽ có một cầu khác đi thẳng đường Phạm Văn Đồng quy mô 4 làn xe hai chiều (cầu vượt sẽ triển khai trong giai đoạn sau của dự án Vành đai 2). Phía dưới nút giao, hầm chui sẽ được xây dựng gồm hai làn, cho xe rẽ phải từ đư

- Lưu nội dung các bài báo thành các file nằm trong folder chuyên mục tương ứng

In [25]:
import os

# Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
sentence = []
#Chữ cái in hoa cảu tiếng việt
uppercase = "AĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXY"

for category in article_url.keys():
    # Tạo folder chuyên mục để chứa các file bài viết
    current_path = 'data/categories/' + category + '/'
    os.mkdir(current_path)

# Lưu các bài viết thành từng file
    count = 0
    for i in range(len(article_url[category])):
        content = _get_content(article_url[category][i])
        if content != '':
            count += 1
            f = open('{}/{}_{}.txt'.format(current_path, category, str(count)), 'w', encoding='utf-8') 
            # Chèn khoảng cách vào sau các dấu câu (., ?, !) nếu không có khoảng trắng hoặc dấu ngoặc kép ngay sau đó
            lines = re.sub(r'(?<=[.?!])(?=[\'"{}])'.format(uppercase), ' ', content)

            # Tách bài viết thành câu (giả sử câu kết thúc bằng ., !, ? và câu mới bắt đầu bằng chữ cái in hoa hoặc chữ số)
            lines = re.split(r'(?<=[.?!])\s*(?=[\'"{}0-9])'.format(uppercase), content)


            # Ghi từng câu trong vào viết vào file
            for line in lines:
                f.write((line + '\n'))
                sentence.append(line)
            f.close
        # Sổ bài viết trong chuyên mục    
        print(category, count)

        sentence = list(set(sentence))
        print("Tổng số câu được tách: ", len(sentence))

thoi-su 1
Tổng số câu được tách:  19
thoi-su 2
Tổng số câu được tách:  69
thoi-su 3
Tổng số câu được tách:  83
thoi-su 4
Tổng số câu được tách:  138
thoi-su 5
Tổng số câu được tách:  165
thoi-su 6
Tổng số câu được tách:  166
thoi-su 7
Tổng số câu được tách:  185
thoi-su 7
Tổng số câu được tách:  185
thoi-su 8
Tổng số câu được tách:  200
thoi-su 9
Tổng số câu được tách:  239
thoi-su 10
Tổng số câu được tách:  270
thoi-su 11
Tổng số câu được tách:  283
thoi-su 12
Tổng số câu được tách:  327
thoi-su 12
Tổng số câu được tách:  327
thoi-su 13
Tổng số câu được tách:  359
thoi-su 14
Tổng số câu được tách:  412
thoi-su 15
Tổng số câu được tách:  482
thoi-su 16
Tổng số câu được tách:  528
thoi-su 17
Tổng số câu được tách:  551
thoi-su 18
Tổng số câu được tách:  596
thoi-su 19
Tổng số câu được tách:  636
thoi-su 20
Tổng số câu được tách:  687
thoi-su 21
Tổng số câu được tách:  719
thoi-su 22
Tổng số câu được tách:  740
thoi-su 23
Tổng số câu được tách:  759
thoi-su 24
Tổng số câu được tách:  797

+ 10 câu dầu tiên tách được

In [26]:
sentence[:10]

['Khi xã hội chưa thể là "ông chủ trả lương đúng và đủ" cho giáo viên, thì sự tôn trọng và thấu hiểu sẽ là nguồn động viên to lớn để thầy cô giáo làm tốt công việc của mình mỗi ngày.',
 'Tuy nhiên, đây mới là lần đầu siêu sao 39 tuổi chơi bóng tại Scotland, và anh tạo ra cơn sốt với khán giả ở đó.',
 'Đồng thời, người sản xuất túi mù cũng đang tận dụng sức hút của Labubu. Ở Việt Nam, dữ liệu từ Metric - nền tảng thống kê thương mại điện tử - cho biết, vào quý II, các mặt hàng liên quan Labubu (chính hãng, xách tay) mang về gần 5,2 tỷ đồng trên Shopee, Lazada, Tiktok Shop.',
 'Theo đề xuất của Bộ Giao thông Vận tải, dự án đường sắt tốc độ cao Bắc Nam có tổng mức đầu tư sơ bộ hơn 1,7 triệu tỷ đồng (tương đương 67,34 tỷ USD), tuyến đường đôi, khổ 1.',
 'Nhiếp ảnh gia Lê Bích 52 tuổi, quê Hà Nội.',
 '000 đồng mỗi kg.',
 'Phong cũng là người thích "túi mù".',
 'Trong đó, tầng một có hai cầu vượt: cầu số 1 gồm hai nhánh (6 làn xe) nằm trên Vành đai 2, vượt qua đường Linh Đông, Phạm Văn Đồng,

<strong>3. Lưu tất cả các câu vào file ngữ liệu </strong>

In [27]:
# File ngữ liệu
file_corpus = open('data/corpus.txt', 'w', encoding='utf-8')

count = 0
for sent in sentence:
    # Xóa ký tự thừa
    sent = re.sub(r'\u200b', '', sent)

    file_corpus.write(sent.strip() + '\n')
    count +=1
file_corpus.close()

print('Số lượng câu trong file ngữ liệu: ', count)

Số lượng câu trong file ngữ liệu:  812
